<a href="https://colab.research.google.com/github/VictorSilvaCamargo/Atividade_Pratica_2_RA3/blob/main/garcom_cozinheiro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import threading
import time
import random

class Garcom(threading.Thread):
    def __init__(self, garcom_id, cozinheiro, garcom_concluido):
        threading.Thread.__init__(self)
        self.garcom_id = garcom_id
        self.cozinheiro = cozinheiro
        self.garcom_concluido = garcom_concluido

    def run(self):
        for i in range(10):
            pedido = f"Pedido {i+1}"
            self.fazer_pedido(pedido)
            self.entregar_pedido()
            tempo_espera = random.uniform(0.3, 0.6)  # Tempo de espera aleatório entre 0.3 e 0.6 segundos
            time.sleep(tempo_espera)
        self.garcom_concluido[self.garcom_id - 1] = True
        self.cozinheiro.finalizar_garcom(self.garcom_id)

    def fazer_pedido(self, pedido):
        with self.cozinheiro.cond:
            while len(self.cozinheiro.pedidos_pendentes) >= 4 or self.cozinheiro.pedido_em_preparo is not None:
                self.cozinheiro.cond.wait()
            self.cozinheiro.pedidos_pendentes.append((pedido, self.garcom_id))
            print(f"Garçom {self.garcom_id} adicionou o pedido '{pedido}' aos pedidos pendentes.\n")
            self.cozinheiro.cond.notify()

    def entregar_pedido(self):
        with self.cozinheiro.cond:
            while self.cozinheiro.pedido_em_preparo is None or self.cozinheiro.pedido_em_preparo[1] != self.garcom_id:
                self.cozinheiro.cond.wait()
            pedido_em_preparo = self.cozinheiro.pedido_em_preparo
            if pedido_em_preparo is not None:
                pedido, garcom_id = pedido_em_preparo
                print(f"Garçom {self.garcom_id} entregou o pedido '{pedido}'\n")
                self.cozinheiro.pedido_em_preparo = None
                self.cozinheiro.cond.notify()

class Cozinheiro(threading.Thread):
    def __init__(self, num_garcons):
        threading.Thread.__init__(self)
        self.pedidos_pendentes = []
        self.pedido_em_preparo = None
        self.cond = threading.Condition()
        self.num_garcons = num_garcons
        self.garcons_concluidos = [False] * num_garcons
        self.cozinha_aberta = True

    def run(self):
        while self.cozinha_aberta:
            with self.cond:
                while len(self.pedidos_pendentes) == 0 and self.pedido_em_preparo is None:
                    self.cond.wait()
                if self.pedido_em_preparo is None and len(self.pedidos_pendentes) > 0:
                    pedido, garcom_id = self.pedidos_pendentes.pop(0)
                    self.pedido_em_preparo = (pedido, garcom_id)
                    print(f"Cozinheiro está preparando o pedido do Garçom {garcom_id}: '{pedido}'\n")
                    time.sleep(1)  # Tempo de preparo do pedido
                    print(f"Cozinheiro preparou o pedido do Garçom {garcom_id}: '{pedido}'\n")
                    self.cond.notify_all()

            # Verifica se todos os garçons concluíram
            if all(self.garcons_concluidos):
                self.cozinha_aberta = False
                print("A cozinha fechou.")

    def finalizar_garcom(self, garcom_id):
        with self.cond:
            self.cond.notify_all()

# Criando o objeto Cozinheiro
cozinheiro = Cozinheiro(num_garcons=5)

# Criando os objetos Garcom
start_time=time.time()
garcons = []
for i in range(5):
    garcom = Garcom(i+1, cozinheiro, cozinheiro.garcons_concluidos)
    garcons.append(garcom)

# Iniciando a thread do cozinheiro
cozinheiro.start()

# Iniciando as threads dos garçons
for garcom in garcons:
    garcom.start()

# Aguardando a finalização das threads dos garçons
for garcom in garcons:
    garcom.join()
end_time=time.time()
print("tempo total:",end_time - start_time)

Garçom 1 adicionou o pedido 'Pedido 1' aos pedidos pendentes.

Cozinheiro está preparando o pedido do Garçom 1: 'Pedido 1'

Cozinheiro preparou o pedido do Garçom 1: 'Pedido 1'

Garçom 1 entregou o pedido 'Pedido 1'

Garçom 3 adicionou o pedido 'Pedido 1' aos pedidos pendentes.

Cozinheiro está preparando o pedido do Garçom 3: 'Pedido 1'

Cozinheiro preparou o pedido do Garçom 3: 'Pedido 1'

Garçom 3 entregou o pedido 'Pedido 1'

Garçom 4 adicionou o pedido 'Pedido 1' aos pedidos pendentes.

Cozinheiro está preparando o pedido do Garçom 4: 'Pedido 1'

Cozinheiro preparou o pedido do Garçom 4: 'Pedido 1'

Garçom 4 entregou o pedido 'Pedido 1'

Garçom 3 adicionou o pedido 'Pedido 2' aos pedidos pendentes.

Garçom 2 adicionou o pedido 'Pedido 1' aos pedidos pendentes.

Garçom 5 adicionou o pedido 'Pedido 1' aos pedidos pendentes.

Cozinheiro está preparando o pedido do Garçom 3: 'Pedido 2'

Cozinheiro preparou o pedido do Garçom 3: 'Pedido 2'

Garçom 3 entregou o pedido 'Pedido 2'

Garçom

In [19]:
import threading
import time
import random

class Garcom(threading.Thread):
    def __init__(self, garcom_id, cozinheiros, garcom_concluido):
        threading.Thread.__init__(self)
        self.garcom_id = garcom_id
        self.cozinheiros = cozinheiros
        self.garcom_concluido = garcom_concluido

    def run(self):
        for i in range(10):
            pedido = f"Pedido {i+1}"
            self.fazer_pedido(pedido)
            self.entregar_pedido()
            tempo_espera = random.uniform(0.3, 0.6)  # Tempo de espera aleatório entre 0.3 e 0.6 segundos
            time.sleep(tempo_espera)
        self.garcom_concluido[self.garcom_id - 1] = True

    def fazer_pedido(self, pedido):
        with Cozinheiro.cond:
            while len(Cozinheiro.pedidos_pendentes) >= 4 or Cozinheiro.pedido_em_preparo is not None:
                Cozinheiro.cond.wait()
            Cozinheiro.pedidos_pendentes.append((pedido, self.garcom_id))
            print(f"Garçom {self.garcom_id} adicionou o pedido '{pedido}' aos pedidos pendentes.\n")
            Cozinheiro.cond.notify()

    def entregar_pedido(self):
        with Cozinheiro.cond:
            while Cozinheiro.pedido_em_preparo is None or Cozinheiro.pedido_em_preparo[1] != self.garcom_id:
                Cozinheiro.cond.wait()
            pedido_em_preparo = Cozinheiro.pedido_em_preparo
            if pedido_em_preparo is not None:
                pedido, garcom_id = pedido_em_preparo
                print(f"Garçom {self.garcom_id} entregou o pedido '{pedido}'\n")
                Cozinheiro.pedido_em_preparo = None
                Cozinheiro.cond.notify()

class Cozinheiro(threading.Thread):
    pedidos_pendentes = []
    pedido_em_preparo = None
    cond = threading.Condition()

    def __init__(self):
        threading.Thread.__init__(self)

    def run(self):
        while True:
            with Cozinheiro.cond:
                while len(Cozinheiro.pedidos_pendentes) == 0 and Cozinheiro.pedido_em_preparo is None:
                    Cozinheiro.cond.wait()
                if Cozinheiro.pedido_em_preparo is None and len(Cozinheiro.pedidos_pendentes) > 0:
                    pedido, garcom_id = Cozinheiro.pedidos_pendentes.pop(0)
                    Cozinheiro.pedido_em_preparo = (pedido, garcom_id)
                    print(f"Cozinheiro está preparando o pedido do Garçom {garcom_id}: '{pedido}'\n")
                    time.sleep(1)  # Tempo de preparo do pedido
                    print(f"Cozinheiro preparou o pedido do Garçom {garcom_id}: '{pedido}'\n")
                    Cozinheiro.cond.notify_all()

            # Verifica se todos os garçons concluíram
            if all(garcom_concluido for garcom_concluido in Garcom.garcom_concluido):
                print("A cozinha fechou.")
                break

# Criando os objetos Garcom
garcons = []
garcons_concluidos = [False] * 5
for i in range(5):
    garcom = Garcom(i+1, Cozinheiro, garcons_concluidos)
    garcons.append(garcom)

# Criando os objetos Cozinheiro
cozinheiros = [Cozinheiro(), Cozinheiro()]

# Iniciando as threads dos garçons
for garcom in garcons:
    garcom.start()

# Iniciando as threads dos cozinheiros
for cozinheiro in cozinheiros:
    cozinheiro.start()

# Aguardando a finalização das threads dos garçons
for garcom in garcons:
    garcom.join()

# Aguardando a finalização das threads dos cozinheiros
for cozinheiro in cozinheiros:
    cozinheiro.join()


Garçom 1 adicionou o pedido 'Pedido 1' aos pedidos pendentes.

Garçom 2 adicionou o pedido 'Pedido 1' aos pedidos pendentes.

Garçom 3 adicionou o pedido 'Pedido 1' aos pedidos pendentes.

Garçom 4 adicionou o pedido 'Pedido 1' aos pedidos pendentes.

Cozinheiro está preparando o pedido do Garçom 1: 'Pedido 1'



Exception in thread Thread-131:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
Exception in thread Thread-132:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "<ipython-input-19-a590e50da22e>", line 62, in run
    self.run()
  File "<ipython-input-19-a590e50da22e>", line 62, in run
AttributeError: type object 'Garcom' has no attribute 'garcom_concluido'
AttributeError: type object 'Garcom' has no attribute 'garcom_concluido'


Cozinheiro preparou o pedido do Garçom 1: 'Pedido 1'

Garçom 1 entregou o pedido 'Pedido 1'

Garçom 1 adicionou o pedido 'Pedido 2' aos pedidos pendentes.



KeyboardInterrupt: ignored